In [1]:
# 세대 설정
# 먼저, 순전파시 '세대'를 설정하는 부분부터 시작하고 그런 다음 역전파 시 최근 세대의 함수부터 꺼내도록 합니다.
# 이렇게 하면 아무리 복잡한 계산 그래프라도 올바른 순서로 역전파가 이루어 집니다.

In [ ]:
# 세대 추가
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} 은(는) 지원하지 않습니다.')
                
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation  + 1 # 부모 세대 + 1
        
        

In [ ]:
# ======[f] ===
#      2세대    ==
#                 ==>{Y} 3세대
#              ==   
#======[C]  ===
#      3세대


In [22]:
class Function(object):
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
            
        self.inputs = inputs
        self.outputs = outputs
        
        return  outputs if len(outputs) > 1 else outputs[0]

In [6]:
# 세대 순으로 sort
generations = [2,  0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function()
    f.generation = g
    funcs.append(f)
    
print([f.generation for  f in funcs])

funcs.sort(key= lambda x : x.generation)
print([f.generation for  f in funcs])

[2, 0, 1, 4, 2]
[0, 1, 2, 2, 4]


[0, 1, 2, 2, 4]

In [54]:
# 세대 추가
class Variable:
    
    
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} 은(는) 지원하지 않습니다.')
                
                
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0
        


    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
            
            
        ### 추가 부분 ==========================================================
        funcs = []
        seen_set = set()
        
        def add_func(f):
            if  f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key = lambda x : x.generation)
                
        add_func(self.creator)
        
        #===========================================================================
        
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            
            for x, gx in zip(f.inputs, gxs):
                
                if x.grad is None: 
                    x.grad = gx
                    
                else:
                    x.grad = x.grad + gx
                    
                if x.creator is not None:
                    add_func(x.creator)
                    

In [55]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        return gy, gy
    
    
def as_array(x):
    if np.isscalar(x):
        return np.array((x))
    return x
def add(x0, x1):
    return Add()(x0, x1)

In [56]:

class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x*gy
        return gx


def square(x):
    return Square()(x)


In [57]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
32.0
